# Federated Learning Pipeline

This annotated notebook explains the pipeline **step by step**, with:
- short introductions before each code cell;
- comments at the top of every cell describing *what it does* and *how to check the result*;
- a **checklist**, **troubleshooting tips**, a **glossary**, and suggested exercises.

> **Learning Objectives**
> - Understand the end‑to‑end flow of a Federated Learning (FL) pipeline: preprocessing → training → evaluation → saving artifacts.
> - Learn to read and adapt components (dataset, model, training loop, metrics).
> - Run experiments (saving results).

> **Prerequisites**
> - Intermediate Python; experience with PyTorch and scikit‑learn.
> - Basic ML concepts: train/val/test split, overfitting, metrics.
> - Introductory knowledge of FL (clients, server/aggregator, training rounds).

> **Mini Glossary (FL)**
> - **Client**: A device/site (e.g., hospital, phone) that trains locally on private data.
> - **Server**:The coordinator distributes the starting model, communicates with clients, collects client updates, and distributes new models.
> - **Aggregator/Aggregation Method**:  Method chosen to aggregate all client models and create the new global model.
> - **Round**: One cycle of local training → sending updates → aggregation.
> - **Aggregation frequency**: Number of epochs between sending weights to the server.
> - **Federated Averaging (FedAvg)**: Standard method to average/aggregate client model weights.
> - **IID Data**: Customer data follows the same distribution across all clients (homogeneity).
> - **Non‑IID Data**: Client data may follow different distributions (heterogeneity).
> - **Global model**: Model sent by the server to all clients
> - **Local model**: Model trained by the client on its data

### Import module
**Environment setup and library imports.**

This section imports all required libraries:
 - Data handling: `pandas`, `numpy`, `sklearn`
 - Model building and training: `torch`, `torch.nn`
 - Visualization: `matplotlib`

In [1]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from copy import deepcopy
import matplotlib.pyplot as plt # New import
import numpy as np # New import

#### Definition of the model, train function, and evaluation.

This part defines:
 - a simple **MLP (Multilayer Perceptron)** model;
 - two helper functions:
   - `train()` → performs one epoch of training;
   - `evaluate()` → computes accuracy and loss on validation/test sets. 

In [2]:
# Model definition
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training and evaluation loops
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    for features, labels in loader:
        features, labels = features.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)

# Evaluation function
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for features, labels in loader:
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            outputs = model(features)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)

#### Data loading and preprocessing

The following helper functions:
 - load CSV data and separate features/labels;
 - normalize features using `StandardScaler`;
 - convert arrays to PyTorch tensors;
 - build DataLoaders for batching.

In [3]:
def load_dataset(train_file, test_file):

    # Load train/test CSVs
    train_data = pd.read_csv(train_file)
    test_data = pd.read_csv(test_file)
    class_names = train_data["Class"].unique()
    print(f"Classes: {class_names}")

    # Feature and label columns
    feature_names = train_data.columns[:-1]
    print(f"Features: {feature_names}")
    
    X_train = train_data[feature_names].values
    y_train = train_data["Class"].values
    X_test = test_data[feature_names].values
    y_test = test_data["Class"].values

    # Encode class labels as integers
    class_map = {label: idx for idx, label in enumerate(class_names)}
    y_train = [class_map[label] for label in y_train]
    y_test = [class_map[label] for label in y_test]

    # Split training data into training and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )
    print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}, Test samples: {len(X_test)}")

    input_dim = X_train.shape[1]
    output_dim = len(class_names)

    return X_train, y_train, X_val, y_val, X_test, y_test, input_dim, output_dim

# Normalize features with mean 0 and std 1
def features_scaling(X_train, X_val, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    print("Features scaled successfully.")
    return X_train_scaled, X_val_scaled, X_test_scaled

# Convert numpy arrays to PyTorch tensors.
def convert_to_tensors(X_train, y_train, X_val, y_val, X_test, y_test):
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor


def create_dataloader(X_tensor, y_tensor, batch_size):
    data = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(data, batch_size=batch_size, shuffle=True)
    return loader



#### Hyperparameters


In [ ]:
HIDDEN_DIM = 64
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PATIENCE = 5  # Early stopping patience

### Pipeline execution

Data loading and data loader creation.

In [ ]:
# Load and preprocess server data
X_train_server, y_train_server, X_val_server, y_val_server, X_test_server, y_test_server, INPUT_DIM, OUTPUT_DIM = load_dataset(
    "health_data_backup/train_health_data_server.csv", "health_data_backup/test_health_data_server.csv")



X_train_server, X_val_server, X_test_server = features_scaling(X_train_server, X_val_server, X_test_server)

X_train_tensor_server, y_train_tensor_server, X_val_tensor_server, y_val_tensor_server, X_test_tensor_server, y_test_tensor_server = convert_to_tensors(X_train_server, y_train_server, X_val_server, y_val_server, X_test_server, y_test_server)

train_loader_server = create_dataloader(X_train_tensor_server, y_train_tensor_server, BATCH_SIZE)
val_loader_server = create_dataloader(X_val_tensor_server, y_val_tensor_server, BATCH_SIZE)
test_loader_server = create_dataloader(X_test_tensor_server, y_test_tensor_server, BATCH_SIZE)

print(f"Device: {DEVICE}, Input dim: {INPUT_DIM}, Output dim: {OUTPUT_DIM}")

FileNotFoundError: [Errno 2] No such file or directory: 'health_data_backup/train_health_data_server.csv'

#### Server training

The server trains a baseline (centralized) model before FL.
 - Uses early stopping to prevent overfitting.
 - Prints training and validation accuracy per epoch.

In [ ]:
####### Train server

# Model, optimizer, and loss
server_model = MLP(INPUT_DIM, OUTPUT_DIM, HIDDEN_DIM).to(DEVICE)
optimizer = torch.optim.Adam(server_model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

# Early stopping setup
best_val_loss = float('inf')
patience_counter = 0


# Training loop with early stopping
print("Starting training...")
for epoch in range(EPOCHS):
    train_loss, train_acc = train(server_model, train_loader_server, optimizer, criterion)
    val_loss, val_acc = evaluate(server_model, val_loader_server, criterion)
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Early stopping check (avoid overfitting)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        print("New best validation loss. Resetting patience counter.")
    else:
        patience_counter += 1
        print(f"No improvement. Patience counter: {patience_counter}/{PATIENCE}")
        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break




# -----Test the model------
test_loss, test_acc = evaluate(server_model, test_loader_server, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# Collect predictions for metrics
server_model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for features, labels in test_loader_server:
        features, labels = features.to(DEVICE), labels.to(DEVICE)
        outputs = server_model(features)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Confusion matrix and other metrics
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

#### Definition of Federated Learning parameters.

The main parameters of FL are defined: **aggregation frequency, aggregation round, number of clients, and data structures** suitable for containing **client data and models**.

These parameters are examples; modify and select them based on your data.

Define FL setup:
 - how often to aggregate (`aggregation_freq`);
 - number of rounds (`aggregation_round`);
 - number of clients and placeholder structures.


In [ ]:
# FL parameters
aggregation_freq = 5
aggregation_round = round(EPOCHS / aggregation_freq)

print(f"Aggregation_round: {aggregation_round}")
clients_number = 5

# FL clients data
train_loader_clients = []
val_loader_clients = []
test_loader_clients = []

models = []

**fed_avg** is FL's **standard** aggregation method. It receives the weights of all models and **averages** each weight. The new model is composed of the **average of all model weights**.

**fed_avg_weighted** is a **variant of fed_avg** that implements **weighted averaging**, taking into account the amount of data from each client.

With the **fed_avg_w** variable, we choose whether to use the weighted version of FedAVG or not.

In [ ]:
def fed_avg(models_weights):
    # Average the weights
    new_state_dict = {}
    for key in models_weights[0].keys():
        new_state_dict[key] = sum([m[key] for m in models_weights]) / len(models_weights)
    return new_state_dict

#######def fed_avg_weighted(<parameters>):
def fed_avg_weighted(models_weights, data_sizes):
    # Weighted average of the weights
    total_size = sum(data_sizes)
    new_state_dict = {}
    for key in models_weights[0].keys():
        new_state_dict[key] = sum([m[key] * (data_sizes[i] / total_size) for i, m in enumerate(models_weights)])
    return new_state_dict


#### Data loading and preprocessing (Clients)

Creation of the global model from the server model: deepcopy(…)

Loading of each client's data and preprocessing.

Creation of models for each client



Each client:
 - loads its local dataset;
 - scales features and converts to tensors;
 - creates DataLoaders;
 - initializes a model with the global weights.

In [ ]:

global_model = deepcopy(server_model)

####### Load and preprocess clients data
# List to store the number of training samples for each client (for weighted FedAvg)
client_data_sizes = [] 

# List to store the data loaders for each client
train_loader_clients = []
val_loader_clients = []
test_loader_clients = []

# Load and preprocess clients data
def load_client_data(client_id):

    train_file = f"health_data_backup/client_{client_id}_train.csv"
    test_file = f"health_data_backup/client_{client_id}_test.csv"

    X_train_client, y_train_client, X_val_client, y_val_client, X_test_client, y_test_client, _, _ = load_dataset(train_file, test_file)

    # Normalize
    X_train_client, X_val_client, X_test_client = features_scaling(X_train_client, X_val_client, X_test_client)

    # Convert to tensors
    X_train_tensor_client, y_train_tensor_client, X_val_tensor_client, y_val_tensor_client, X_test_tensor_client, y_test_tensor_client = convert_to_tensors(
        X_train_client, y_train_client, X_val_client, y_val_client, X_test_client, y_test_client)

    # Create data loaders for the client
    train_loader_client = create_dataloader(X_train_tensor_client, y_train_tensor_client, BATCH_SIZE)
    val_loader_client = create_dataloader(X_val_tensor_client, y_val_tensor_client, BATCH_SIZE)
    test_loader_client = create_dataloader(X_test_tensor_client, y_test_tensor_client, BATCH_SIZE)

    # Return train size as well
    return train_loader_client, val_loader_client, test_loader_client, len(X_train_client)

# Loop to load data for all clients
for i in range(clients_number):
    train_loader, val_loader, test_loader, train_size = load_client_data(i)
    train_loader_clients.append(train_loader)
    val_loader_clients.append(val_loader)
    test_loader_clients.append(test_loader)
    client_data_sizes.append(train_size)
    
print(f"Client training data sizes: {client_data_sizes}")





####### Create models for each client with server weights
def create_client_model():
    client_model = MLP(INPUT_DIM, OUTPUT_DIM, HIDDEN_DIM).to(DEVICE)
    # The initial global model state is already from the trained server model.
    client_model.load_state_dict(deepcopy(global_model.state_dict())) 
    return client_model

# Initialize client models (Task 3.1 B, C)
models = [create_client_model() for _ in range(clients_number)]

#### Now we need to define the core of the FL, the aggregation and training cycles.

Use two nested cycles.

You can draw **inspiration** from server training to train each client.

For each round, you need to **aggregate the models** with the chosen function (pass the correct parameters based on the function).

Also remember to **evaluate** the global model obtained and print the metrics.

Then start a new round, and each client works with the new models.


This is the **core FL training process**:
 - Each client trains locally for several epochs;
 - Local weights are collected;
 - Global model is updated by averaging;
 - Metrics are logged for analysis.

In [ ]:
####### Tip: Setup a structure to store all values

def run_federated_training(weighted=False):

    global_model = MLP(INPUT_DIM, OUTPUT_DIM, HIDDEN_DIM).to(DEVICE)

    # Dictionnaire pour stocker les métriques par round (acc/loss)
    global_metrics = {
        'round': [], 
        'acc': [], 
        'loss': [], 
        'global_weights': [], 
        'per_client_acc': [] # To store local val accuracies for final bar plot
    }

    criterion = nn.CrossEntropyLoss()

    for r in range(aggregation_round):
        local_weights = []
        local_losses = []
        local_val_accs_current_round = [] # Store val accs for this round

        for i in range(clients_number):
            # Start with the current global model state
            local_model = deepcopy(global_model)
            optimizer = torch.optim.Adam(local_model.parameters(), lr=LEARNING_RATE)

            for _ in range(aggregation_freq):
                train_loss, train_acc = train(local_model, train_loader_clients[i], optimizer, criterion)

            local_loss, local_acc = evaluate(local_model, val_loader_clients[i], criterion)

            # Stockage des résultats pour l'agrégation
            local_losses.append(local_loss)
            local_val_accs_current_round.append(local_acc)
            local_weights.append(deepcopy(local_model.state_dict()))

        # Agrégation des modèles
        if weighted:
            new_state_dict = fed_avg_weighted(local_weights, client_data_sizes)
        else:
            new_state_dict = fed_avg(local_weights)
            
        # Update the global model with aggregated weights
        global_model.load_state_dict(new_state_dict)

        # Log metrics for the round
        global_loss = np.mean(local_losses)
        global_acc = np.mean(local_val_accs_current_round)

        global_metrics['round'].append(r+1)
        global_metrics['loss'].append(global_loss)
        global_metrics['acc'].append(global_acc)


        # Store a copy of the new global weights
        global_metrics['global_weights'].append(deepcopy(global_model.state_dict())) 
        # Store local validation accuracies for the current round
        global_metrics['per_client_acc'].append(local_val_accs_current_round)

       
        print(f"Round {r+1}/{aggregation_round} | {'Weighted' if weighted else 'Standard'} FedAvg | Avg Val Acc={global_acc:.4f} | Avg Val Loss={global_loss:.4f}")
    return global_model, global_metrics


print("\n=== Standard FedAvg ===")
model_fedavg, metrics_fedavg = run_federated_training(weighted=False)

print("\n=== Weighted FedAvg ===")
model_fedavg_w, metrics_fedavg_w = run_federated_training(weighted=True)


# Model validation

Test the model and print the metrics.

In [ ]:
####### Test the aggregated model

print("\n--- Final Model Validation ---")

# Final Standard FedAvg Model Evaluation
final_global_model_std = deepcopy(global_model)
final_global_model_std.load_state_dict(metrics_fedavg['global_weights'][-1])  # Load the last computed weights for standard FedAvg

test_loss_std, test_acc_std = evaluate(final_global_model_std, test_loader_server, criterion)
print(f"Standard FedAvg Final Test Loss: {test_loss_std:.4f}, Final Test Accuracy: {test_acc_std:.4f}")

# Final Weighted FedAvg Model Evaluation
final_global_model_w = deepcopy(global_model)
final_global_model_w.load_state_dict(metrics_fedavg_w['global_weights'][-1])  # Load the last computed weights for weighted FedAvg

test_loss_w, test_acc_w = evaluate(final_global_model_w, test_loader_server, criterion)
print(f"Weighted FedAvg Final Test Loss: {test_loss_w:.4f}, Final Test Accuracy: {test_acc_w:.4f}")

# Re-evaluate the final Weighted FedAvg model to get detailed metrics (like server training)
y_true = []
y_pred = []
final_global_model_w.eval()
with torch.no_grad():
    for features, labels in test_loader_server:
        features, labels = features.to(DEVICE), labels.to(DEVICE)
        outputs = final_global_model_w(features)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

conf_matrix = confusion_matrix(y_true, y_pred)
print("\nWeighted FedAvg Final Model Confusion Matrix:")
print(conf_matrix)

f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print(f"Weighted FedAvg Final F1 Score: {f1:.4f}")
print(f"Weighted FedAvg Final Precision: {precision:.4f}")
print(f"Weighted FedAvg Final Recall: {recall:.4f}")


# Graphs

Now all that's left is to print a few graphs to better illustrate the results.

In [ ]:
#########
plt.figure(figsize=(10,5))
plt.plot(metrics_fedavg['round'], metrics_fedavg['acc'], label='Standard FedAvg', marker='o')
plt.plot(metrics_fedavg_w['round'], metrics_fedavg_w['acc'], label='Weighted FedAvg', marker='s')
plt.xlabel("Round")
plt.ylabel("Global Accuracy")
plt.title("Global Accuracy vs Communication Rounds")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10,5))
plt.bar([f"Client {i+1}" for i in range(clients_number)], client_data_sizes)
plt.title("Data Distribution per Client")
plt.ylabel("Number of Samples")
plt.show()



plt.figure(figsize=(10,5))
plt.plot(metrics_fedavg['round'], metrics_fedavg['loss'], label='Standard FedAvg', marker='o')
plt.plot(metrics_fedavg_w['round'], metrics_fedavg_w['loss'], label='Weighted FedAvg', marker='s')
plt.xlabel("Round")
plt.ylabel("Global Loss")
plt.title("Global Loss vs. Communication Rounds")
plt.legend()
plt.grid(True)
plt.show()


##  Interpretation paragraph

 - **Standard FedAvg** treats all clients equally regardless of dataset size, which may bias results toward smaller clients.
 - **Weighted FedAvg** gives larger clients more influence, leading to more stable and fairer global models.
 - In most experiments, **Weighted FedAvg** converges faster and achieves higher overall accuracy.


A completed Jupyter Notebook with:
- Correct implementation of both FedAvg versions.
- Code for all required plots.
- Brief textual comments explaining your observations.
- A short paragraph comparing the behavior of FedAvg and Weighted FedAvg (e.g., which converges faster, which is fairer across clients, etc.) and your interpretation.
